In [1]:
import numpy as np
import jax
from numpy.random import multivariate_normal as mvn
from approx_post import ApproximateDistribution, JointDistribution, reverse_kl, forward_kl

In [2]:
def create_data(model, true_theta, noise_cov, num_samples):
    mean = model(true_theta)
    samples = mvn(mean, noise_cov, num_samples)
    return samples.reshape(num_samples, -1)

In [3]:
# First, let's define a model:
ndim = 3
model = lambda theta: theta**2
model_grad = jax.vmap(jax.jacfwd(model), in_axes=0)

In [4]:
# Create artificial data:
true_theta = 5*np.random.rand(ndim)
noise_cov = 0.01*np.identity(ndim)
num_samples = 3
data = create_data(model, true_theta, noise_cov, num_samples)
print(f'True theta: \n {true_theta}')
print(f'True x = model(theta): \n {model(true_theta)}')
print(f'Observations x_obs = model(theta) + noise: \n {data}')

True theta: 
 [2.40320681 1.52681422 1.96291679]
True x = model(theta): 
 [5.77540298 2.33116165 3.85304233]
Observations x_obs = model(theta) + noise: 
 [[5.87051362 2.44448495 3.94581996]
 [5.7086499  2.21821999 3.93203575]
 [5.71342434 2.26671093 3.76674922]]


In [5]:
# Create Gaussian approximate distribution:
approx = ApproximateDistribution.gaussian(ndim)

In [6]:
# Create Joint distribution from forward model:
prior_mean = np.zeros(ndim)
prior_cov = np.identity(ndim)
joint = JointDistribution.from_model(data, model, noise_cov, prior_mean, prior_cov, model_grad)

In [9]:
# Fit sddistribution to reverse KL divergence:
results_dict = reverse_kl.fit(approx, joint, use_reparameterisation=True, verbose=True, num_samples=1000)

Now fitting approximate distribution by minimising reverse KL divergence.
Iteration 1:
   Loss = 13488595.0 
   Phi = {'chol_diag': DeviceArray([9.9, 9.9, 9.9], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.1,  0.1,  0.1], dtype=float32), 'mean': DeviceArray([-0.1       ,  0.1       , -0.10000001], dtype=float32)}
Iteration 2:
   Loss = 13066723.0 
   Phi = {'chol_diag': DeviceArray([9.80028 , 9.799919, 9.800565], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.15455326,  0.0829065 ,  0.07807915], dtype=float32), 'mean': DeviceArray([-0.05385482,  0.02923425, -0.16014206], dtype=float32)}
Iteration 3:
   Loss = 12027097.0 
   Phi = {'chol_diag': DeviceArray([9.70159 , 9.700181, 9.700792], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.19713336,  0.04039015,  0.01198923], dtype=float32), 'mean': DeviceArray([-0.06028127, -0.0078432 , -0.2182903 ], dtype=float32)}
Iteration 4:
   Loss = 12747807.0 
   Phi = {'chol_diag': DeviceArray([9.603735, 9.599959, 9.600634], dtype=float32)

Iteration 30:
   Loss = 4228452.0 
   Phi = {'chol_diag': DeviceArray([7.3383713, 7.2700515, 7.2840657], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.02403718,  0.11697512,  0.06182062], dtype=float32), 'mean': DeviceArray([ 0.04106906, -0.257508  ,  0.19831386], dtype=float32)}
Iteration 31:
   Loss = 3697585.0 
   Phi = {'chol_diag': DeviceArray([7.2629433, 7.195138 , 7.2107186], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.04546546,  0.0982356 ,  0.07411161], dtype=float32), 'mean': DeviceArray([ 0.03557166, -0.23925522,  0.1881381 ], dtype=float32)}
Iteration 32:
   Loss = 3465987.0 
   Phi = {'chol_diag': DeviceArray([7.1880093, 7.121482 , 7.1394396], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.06139163,  0.08367477,  0.08316988], dtype=float32), 'mean': DeviceArray([ 0.03242374, -0.21876283,  0.16914447], dtype=float32)}
Iteration 33:
   Loss = 3348369.75 
   Phi = {'chol_diag': DeviceArray([7.11477 , 7.049393, 7.068563], dtype=float32), 'chol_lowerdiag': DeviceAr

Iteration 59:
   Loss = 1251688.5 
   Phi = {'chol_diag': DeviceArray([5.636788 , 5.5349045, 5.6084476], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.03291577,  0.0588314 , -0.04320851], dtype=float32), 'mean': DeviceArray([-0.06637874, -0.1028661 , -0.15204325], dtype=float32)}
Iteration 60:
   Loss = 1208672.5 
   Phi = {'chol_diag': DeviceArray([5.593718 , 5.4899325, 5.5655594], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.04620408,  0.06664146, -0.03932977], dtype=float32), 'mean': DeviceArray([-0.06467816, -0.11255858, -0.14585532], dtype=float32)}
Iteration 61:
   Loss = 1313731.5 
   Phi = {'chol_diag': DeviceArray([5.550734 , 5.445563 , 5.5234346], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.0600598 ,  0.07165306, -0.03980357], dtype=float32), 'mean': DeviceArray([-0.06319935, -0.11937749, -0.13665314], dtype=float32)}
Iteration 62:
   Loss = 1069796.125 
   Phi = {'chol_diag': DeviceArray([5.5092154, 5.402146 , 5.481989 ], dtype=float32), 'chol_lowerdiag': Devi

Iteration 88:
   Loss = 507936.09375 
   Phi = {'chol_diag': DeviceArray([4.674088 , 4.5272017, 4.617197 ], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.09562985, -0.06435157,  0.03461273], dtype=float32), 'mean': DeviceArray([-0.00292352, -0.045761  ,  0.12409248], dtype=float32)}
Iteration 89:
   Loss = 574026.9375 
   Phi = {'chol_diag': DeviceArray([4.6486397, 4.500638 , 4.590156 ], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.09358332, -0.07027133,  0.03022184], dtype=float32), 'mean': DeviceArray([ 0.00530808, -0.04680131,  0.11736035], dtype=float32)}
Iteration 90:
   Loss = 514214.53125 
   Phi = {'chol_diag': DeviceArray([4.6233377, 4.4746637, 4.5636215], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.09053762, -0.07578923,  0.02562445], dtype=float32), 'mean': DeviceArray([ 0.01105113, -0.04760285,  0.11113584], dtype=float32)}
Iteration 91:
   Loss = 448939.78125 
   Phi = {'chol_diag': DeviceArray([4.599014 , 4.4489436, 4.5376472], dtype=float32), 'chol_lowerdi

Iteration 117:
   Loss = 297146.6875 
   Phi = {'chol_diag': DeviceArray([4.047919 , 3.8633509, 3.9728856], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.06559094,  0.07372686, -0.0229885 ], dtype=float32), 'mean': DeviceArray([ 0.02708839, -0.02530622, -0.05201748], dtype=float32)}
Iteration 118:
   Loss = 288847.46875 
   Phi = {'chol_diag': DeviceArray([4.029602 , 3.845096 , 3.9549851], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.06469465,  0.07530625, -0.02163678], dtype=float32), 'mean': DeviceArray([ 0.03003438, -0.02459053, -0.0582595 ], dtype=float32)}
Iteration 119:
   Loss = 276557.6875 
   Phi = {'chol_diag': DeviceArray([4.011231 , 3.8271897, 3.937452 ], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.06272389,  0.07304724, -0.01892442], dtype=float32), 'mean': DeviceArray([ 0.03063217, -0.023553  , -0.06644365], dtype=float32)}
Iteration 120:
   Loss = 246944.453125 
   Phi = {'chol_diag': DeviceArray([3.993297 , 3.8094535, 3.9203324], dtype=float32), 'chol_low

...

Iteration 868:
   Loss = 5464.4794921875 
   Phi = {'chol_diag': DeviceArray([1.4941368, 1.1695603, 1.3317206], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00667111,  0.00575524, -0.00127242], dtype=float32), 'mean': DeviceArray([0.02992694, 0.00723294, 0.0038694 ], dtype=float32)}
Iteration 869:
   Loss = 5752.82421875 
   Phi = {'chol_diag': DeviceArray([1.4936628, 1.1688954, 1.331132 ], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00660723,  0.00588571, -0.00171704], dtype=float32), 'mean': DeviceArray([0.03009895, 0.00706831, 0.00415016], dtype=float32)}
Iteration 870:
   Loss = 5665.3662109375 
   Phi = {'chol_diag': DeviceArray([1.4931995, 1.1682272, 1.3305405], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00657077,  0.00593095, -0.00204566], dtype=float32), 'mean': DeviceArray([0.03040317, 0.0069011 , 0.00417105], dtype=float32)}
Iteration 871:
   Loss = 5632.90576171875 
   Phi = {'chol_diag': DeviceArray([1.4927465, 1.1675515, 1.3299638], dtype=float32), 'chol_

Iteration 897:
   Loss = 5439.17919921875 
   Phi = {'chol_diag': DeviceArray([1.4818108, 1.1510295, 1.3155849], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00465819,  0.00500431, -0.00248211], dtype=float32), 'mean': DeviceArray([ 0.02672518, -0.00074305,  0.00489552], dtype=float32)}
Iteration 898:
   Loss = 5653.05517578125 
   Phi = {'chol_diag': DeviceArray([1.4813918, 1.1504297, 1.3150241], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00448964,  0.00478795, -0.00231874], dtype=float32), 'mean': DeviceArray([ 0.02660285, -0.00090111,  0.00525032], dtype=float32)}
Iteration 899:
   Loss = 5921.5009765625 
   Phi = {'chol_diag': DeviceArray([1.4809124, 1.1498337, 1.3144678], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00434176,  0.00456612, -0.00207869], dtype=float32), 'mean': DeviceArray([ 0.02624285, -0.00107812,  0.00558085], dtype=float32)}
Iteration 900:
   Loss = 5595.0625 
   Phi = {'chol_diag': DeviceArray([1.480456 , 1.1492283, 1.3139118], dtype=float32), 

Iteration 926:
   Loss = 5802.24169921875 
   Phi = {'chol_diag': DeviceArray([1.4698639, 1.133653 , 1.3009508], dtype=float32), 'chol_lowerdiag': DeviceArray([0.00139174, 0.00345867, 0.00675601], dtype=float32), 'mean': DeviceArray([ 0.0190408 , -0.00942199,  0.01481012], dtype=float32)}
Iteration 927:
   Loss = 5653.7412109375 
   Phi = {'chol_diag': DeviceArray([1.4694972, 1.1330014, 1.3004726], dtype=float32), 'chol_lowerdiag': DeviceArray([0.00110591, 0.00347164, 0.006864  ], dtype=float32), 'mean': DeviceArray([ 0.01872448, -0.0093546 ,  0.01506198], dtype=float32)}
Iteration 928:
   Loss = 5568.90771484375 
   Phi = {'chol_diag': DeviceArray([1.4691335, 1.1323394, 1.3000201], dtype=float32), 'chol_lowerdiag': DeviceArray([0.00089609, 0.0034325 , 0.00705801], dtype=float32), 'mean': DeviceArray([ 0.01803995, -0.0093399 ,  0.01518686], dtype=float32)}
Iteration 929:
   Loss = 5571.17236328125 
   Phi = {'chol_diag': DeviceArray([1.4687603, 1.1316735, 1.2995732], dtype=float32), 'c

Iteration 955:
   Loss = 5526.78662109375 
   Phi = {'chol_diag': DeviceArray([1.4601882, 1.1166257, 1.2873026], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.0005104 ,  0.00079331,  0.00533189], dtype=float32), 'mean': DeviceArray([ 0.01854331, -0.00648943,  0.00089503], dtype=float32)}
Iteration 956:
   Loss = 5393.3984375 
   Phi = {'chol_diag': DeviceArray([1.4598582, 1.1160994, 1.2868564], dtype=float32), 'chol_lowerdiag': DeviceArray([-0.00029343,  0.00067245,  0.00510439], dtype=float32), 'mean': DeviceArray([ 0.01891613, -0.00613074,  0.00028946], dtype=float32)}
Iteration 957:
   Loss = 5564.4423828125 
   Phi = {'chol_diag': DeviceArray([1.4595313, 1.1155715, 1.2864001], dtype=float32), 'chol_lowerdiag': DeviceArray([-7.6078693e-05,  6.1217340e-04,  4.9153636e-03], dtype=float32), 'mean': DeviceArray([ 0.01922624, -0.00577237, -0.00042445], dtype=float32)}
Iteration 958:
   Loss = 5766.00634765625 
   Phi = {'chol_diag': DeviceArray([1.4591887, 1.1150324, 1.2859437], dtyp

Iteration 984:
   Loss = 5438.05712890625 
   Phi = {'chol_diag': DeviceArray([1.4524078, 1.1019442, 1.2751305], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00222672, -0.00574544,  0.00208283], dtype=float32), 'mean': DeviceArray([ 0.02162471, -0.00072461, -0.01015406], dtype=float32)}
Iteration 985:
   Loss = 5344.140625 
   Phi = {'chol_diag': DeviceArray([1.4522157, 1.1014354, 1.2747293], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00204357, -0.0062075 ,  0.00210873], dtype=float32), 'mean': DeviceArray([ 0.02164211, -0.0004869 , -0.01019295], dtype=float32)}
Iteration 986:
   Loss = 5740.56787109375 
   Phi = {'chol_diag': DeviceArray([1.4519788, 1.1008935, 1.2743475], dtype=float32), 'chol_lowerdiag': DeviceArray([ 0.00186233, -0.00639039,  0.00205695], dtype=float32), 'mean': DeviceArray([ 2.1446610e-02, -9.4111369e-05, -1.0172525e-02], dtype=float32)}
Iteration 987:
   Loss = 5426.6142578125 
   Phi = {'chol_diag': DeviceArray([1.4517368, 1.1003572, 1.2739727], dtype